**Importing Libraries**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras.utils as ks    
import os
import matplotlib.pyplot as plt
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint

**Download and Extract in Google Colab**

In [ ]:
!gdown 1bcx0UBEaofj4Rj9d6yZm0R7Ycf3-vsQX
!tar -xvf "/content/PlantVillage.tar" -C "/content/"

**Extract downloaded images file**

In [ ]:
# Used while training

# import shutil

# datafolder = "PlantVillage/"
# if os.path.exists(datafolder):
#     shutil.rmtree(datafolder)
# !tar -xf "emmarex_plantdisease.zip" -C "./" "PlantVillage/Potato*"

**Modifies the current folder of images with same names**

In [ ]:
# Used while training

# datafolder = "./PlantVillage/"
# for folder_name in os.listdir(datafolder):
#     count=1
#     for file_name in os.listdir(datafolder+folder_name+"/"):
#         source = datafolder+folder_name+"/"+file_name
#         destination = datafolder+folder_name+"/n_"+str(count)+".jpg"
#         os.rename(source, destination)
#         count += 1

**Copies the modified images from previous folder to Dataset folder and splitting them into Training, Validation and Testing Sets**

In [ ]:
# Used while training

# import random

# def chooseRandomValue(randomInt,dir_len):
#     num = random.randint(1,dir_len)
#     if num not in randomInt:
#         randomInt.append(num)
#         return num
#     return chooseRandomValue(randomInt,dir_len)

# def splitTrainTestValid(src,val,dst,set):
#     shutil.copy(src+"/n_"+str(val)+".jpg",dst+"n_"+str(set)+".jpg")

j=0
datafolder = "./PlantVillage/"
folder = "./Dataset/"
test = os.path.join(folder,"Test/")
train = os.path.join(folder,"Train/")
valid = os.path.join(folder,"Valid/")
# if os.path.exists(folder):
#     shutil.rmtree(folder)

# categories = ["EB\\","Healthy\\","LB\\"]
# for i in categories:
#     os.makedirs(test+i)
#     os.makedirs(train+i)
#     os.makedirs(valid+i)

# train_size = 0.8
# valid_size = 0.1

# for folder_name in os.listdir(datafolder):
#     randomInt = []
#     dir_len = len(os.listdir(datafolder+folder_name))
#     for i in range(1,dir_len+1):
#         num = chooseRandomValue(randomInt,dir_len)
#         if i<=round(train_size*dir_len):
#             splitTrainTestValid(datafolder+folder_name,num,train+str(categories[j]),i)
#         elif i<=round((train_size+valid_size)*dir_len) and i>round(train_size*dir_len):
#             splitTrainTestValid(datafolder+folder_name,num,valid+str(categories[j]),i)
#         elif i<=dir_len and i>round((train_size+valid_size)*dir_len):
#             splitTrainTestValid(datafolder+folder_name,num,test+str(categories[j]),i)
#     j+=1

**Define training, validation and testing datasets**

In [ ]:
labels = "inferred"
label_mode = "categorical"
batch_size = 16
image_size = (256,256)

train_ds = ks.image_dataset_from_directory(
    train,
    image_size=image_size,
    labels=labels,
    label_mode=label_mode,
    batch_size=batch_size,
)
valid_ds = ks.image_dataset_from_directory(
    valid,
    labels=labels,
    label_mode=label_mode,
    batch_size=batch_size,
    image_size=image_size,
)
test_ds = ks.image_dataset_from_directory(
    test,
    labels=labels,
    label_mode=label_mode,
    batch_size=batch_size//batch_size,
    image_size=image_size,
)

**Display random images from training dataset**

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1): 
    for i in range(16):
        ax = plt.subplot(4, 4, i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis("off")

**ResNet50 Identity Block**

In [ ]:
def identity_block(X, filter):
    X_skip = X
    #Layer 1
    X = Conv2D(
        filters=filter[0],
        kernel_size=1,
        padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    #Layer 2
    X = Conv2D(
        filters=filter[1],
        kernel_size=3,
        padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    #Layer 3
    X = Conv2D(
        filters=filter[2],
        kernel_size=1,
        padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    # SKIP CONNECTION
    X = Add()([X, X_skip])
    X = Activation('relu')(X)
    return X

**ResNet50 Convolutional Block**

In [ ]:
def convolutional_block(X, filter):
    X_skip = X
    #Layer 1
    X = Conv2D(
        filters=filter[0],
        kernel_size=1,
        strides=2,
        padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    #Layer 2
    X = Conv2D(
        filters=filter[1],
        kernel_size=3,
        padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    #Layer 3
    X = Conv2D(
        filters=filter[2],
        kernel_size=1,
        padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    #Layer 4
    X_skip = Conv2D(
        filters=filter[2],
        kernel_size=1,
        strides=2,
        padding='same')(X_skip)
    X_skip = BatchNormalization(axis=3)(X_skip)
    #SKIP CONNECTION
    X = Add()([X, X_skip])
    X = Activation('relu')(X)
    return X

**ResNet50 model block**

In [ ]:
def ResNet50(input_shape):
    X_input = X = Input(input_shape)
    X = ZeroPadding2D(padding=3)(X)
    #Layer 1
    X = Conv2D(
        filters=64,
        kernel_size=7,
        strides=2,
        padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(
        pool_size=3,
        strides=2,
        padding='same')(X)
    #Layer 2
    X = convolutional_block(X, [64, 64, 256])
    X = identity_block(X, [64, 64, 256])
    X = identity_block(X, [64, 64, 256])
    #Layer 3
    X = convolutional_block(X, [128, 128, 512])
    X = identity_block(X, [128, 128, 512])
    X = identity_block(X, [128, 128, 512])
    X = identity_block(X, [128, 128, 512])
    #Layer 4
    X = convolutional_block(X, [256, 256, 1024])
    X = identity_block(X, [256, 256, 1024])
    X = identity_block(X, [256, 256, 1024])
    X = identity_block(X, [256, 256, 1024])
    X = identity_block(X, [256, 256, 1024])
    X = identity_block(X, [256, 256, 1024])
    #Layer 5
    X = convolutional_block(X, [512, 512, 2048])
    X = identity_block(X, [512, 512, 2048])
    X = identity_block(X, [512, 512, 2048])
    X = AveragePooling2D(
        pool_size=2,
        padding='same')(X)
    model = Model(
        inputs=X_input,
        outputs=X)
    return model

In [ ]:
base_model = ResNet50(image_size + (3,))
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
headModel = base_model.output
headModel = Flatten()(headModel)
headModel = Dense(1024, activation='relu')(headModel)
headModel = Dense(512, activation='relu')(headModel)
headModel = Dense(256, activation='relu')(headModel)
headModel = Dense(128, activation='relu')(headModel)
headModel = Dense(3, activation='softmax')(headModel)
model = Model(inputs=base_model.input, outputs=headModel)

**A Brief Summary of the implemented model**

In [ ]:
# Used while training

# model.summary()

**Plotting of the above implemented model**

In [ ]:
# Used while training

# from keras.utils import plot_model

# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

**Already compiled and trained model**

In [ ]:
model.load_weights('./model.h5')

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

**For Training only**

In [ ]:
# Used while training

# epoch = 200

# mc = ModelCheckpoint(
#     f'./final_model.h5',
#     monitor='val_accuracy',
#     save_best_only=True,
#     verbose=1,
#     mode='max'
# )

# history = model.fit(
#     train_ds,
#     epochs=epoch,
#     validation_data=valid_ds,
#     callbacks=[mc]
# )

**For Testing unseen data only**

In [ ]:
model.evaluate(test_ds)

**Display Accuracy Graph**

In [ ]:
plt.figure(figsize=(8, 8))

epochs_range= range(epoch)

plt.plot(epochs_range, history.history['accuracy'], label="Training Accuracy")

plt.plot(epochs_range, history.history['val_accuracy'], label="Validation Accuracy")

plt.axis(ymin=0.4,ymax=1)

plt.grid()

plt.title('Model Accuracy')

plt.ylabel('Accuracy')

plt.xlabel('Epochs')

plt.legend(['train', 'validation'])

plt.savefig('output-plot.png')

plt.show()

**Display Loss Graph**

In [ ]:
plt.figure(figsize=(8, 8))

epochs_range= range(epoch)

plt.plot( epochs_range, history.history['loss'], label="Training Loss")

plt.plot(epochs_range, history.history['val_loss'], label="Validation Loss")

plt.axis(ymin=0.0,ymax=1.0)

plt.grid()

plt.title('Model Loss')

plt.ylabel('Loss')

plt.xlabel('Epochs')

plt.legend(['train', 'validation'])

plt.savefig('output-plot-loss.png')

plt.show()